# (EX) Electronic medical record (as a SQL refresher)
This example utilizes a subset of the [MIMIC III demo](https://physionet.org/content/mimiciii-demo/1.4/) dataset in illustrating the use of database using DuckDB.

*Quick notes about DuckDB:*  

- DuckDB is a relational database for analytics processing (i.e., OLAP)
- DuckDB is columnar-oriented
- DuckDB scales reasonably for *relatively* large datasets and works well for local development

For MIMIC-III, here is a [full summary](https://mit-lcp.github.io/mimic-schema-spy/) where you can understand the relationship between tables.

In [12]:
!pip install duckdb==1.2.2
import duckdb

In [24]:
tables = [
    'ADMISSIONS',
    'ICUSTAYS',
    'PATIENTS',
    'PROCEDURES_ICD',
    'D_ICD_PROCEDURES',
    'DRGCODES',
    'PRESCRIPTIONS'
]

conn = duckdb.connect()
for table_name in tables:
    conn.execute(f"CREATE TABLE {table_name} AS SELECT * FROM read_csv_auto('{table_name}.csv')")

conn.sql('SHOW TABLES;')

┌──────────────────┐
│       name       │
│     varchar      │
├──────────────────┤
│ ADMISSIONS       │
│ DRGCODES         │
│ D_ICD_PROCEDURES │
│ ICUSTAYS         │
│ PATIENTS         │
│ PRESCRIPTIONS    │
│ PROCEDURES_ICD   │
└──────────────────┘

All cols in all tables:

In [22]:
for table_name in tables:
  print(f"Columns in table: {table_name}")
  result = conn.execute(f"PRAGMA table_info({table_name});").fetchall()
  for row in result:
    print(row)
  print("-" * 20)

Columns in table: ADMISSIONS
(0, 'row_id', 'BIGINT', False, None, False)
(1, 'subject_id', 'BIGINT', False, None, False)
(2, 'hadm_id', 'BIGINT', False, None, False)
(3, 'admittime', 'TIMESTAMP', False, None, False)
(4, 'dischtime', 'TIMESTAMP', False, None, False)
(5, 'deathtime', 'TIMESTAMP', False, None, False)
(6, 'admission_type', 'VARCHAR', False, None, False)
(7, 'admission_location', 'VARCHAR', False, None, False)
(8, 'discharge_location', 'VARCHAR', False, None, False)
(9, 'insurance', 'VARCHAR', False, None, False)
(10, 'language', 'VARCHAR', False, None, False)
(11, 'religion', 'VARCHAR', False, None, False)
(12, 'marital_status', 'VARCHAR', False, None, False)
(13, 'ethnicity', 'VARCHAR', False, None, False)
(14, 'edregtime', 'TIMESTAMP', False, None, False)
(15, 'edouttime', 'TIMESTAMP', False, None, False)
(16, 'diagnosis', 'VARCHAR', False, None, False)
(17, 'hospital_expire_flag', 'BIGINT', False, None, False)
(18, 'has_chartevents_data', 'BIGINT', False, None, False)
-

In [48]:

conn.sql(
    """
    WITH drug_ethnicity AS (
        SELECT
            a.ethnicity,
            p.drug AS drug_name,
            COUNT(*) AS prescription_count,
            ROW_NUMBER() OVER (PARTITION BY a.ethnicity ORDER BY COUNT(*) DESC) AS rank
        FROM PRESCRIPTIONS p
        JOIN ADMISSIONS a ON p.hadm_id = a.hadm_id
        WHERE p.drug IS NOT NULL
        GROUP BY a.ethnicity, p.drug
    )
    SELECT
        ethnicity,
        drug_name AS top_drug,
        prescription_count
    FROM drug_ethnicity
    WHERE rank = 1
    ORDER BY prescription_count DESC;
    """
)

┌──────────────────────────────────────────────────────────┬──────────────────────┬────────────────────┐
│                        ethnicity                         │       top_drug       │ prescription_count │
│                         varchar                          │       varchar        │       int64        │
├──────────────────────────────────────────────────────────┼──────────────────────┼────────────────────┤
│ WHITE                                                    │ Potassium Chloride   │                381 │
│ HISPANIC/LATINO - PUERTO RICAN                           │ 0.9% Sodium Chloride │                 86 │
│ BLACK/AFRICAN AMERICAN                                   │ Insulin              │                 38 │
│ UNKNOWN/NOT SPECIFIED                                    │ D5W                  │                 37 │
│ HISPANIC OR LATINO                                       │ 5% Dextrose          │                 28 │
│ UNABLE TO OBTAIN                                     

<br>
ADMISSIONS have an ethnicity and PATIENTS have a drug listed. These were joined and grouped by ethnicity and drug. The drug with the highest count is listed


<br>
<br>
NS and 0.9% Sodium Chloride are the same. These are both normal saline solution. This indicates use of IVs. One outlier is Insulin for Black/African Americans. D5W and 5% Dextrose are both carbohydrate solutions. This analysis did not combine similar drugs or drugs under different names. This also did not group any ethnicities together.



In [26]:
conn.sql(
    """
    WITH patient_ages AS (
        SELECT
            p.subject_id,
            a.hadm_id,
            FLOOR(DATEDIFF('year', p.dob, a.admittime)) AS admission_age
        FROM PATIENTS p
        JOIN ADMISSIONS a ON p.subject_id = a.subject_id
    ),
    age_groups AS (
        SELECT
            subject_id,
            hadm_id,
            CASE
                WHEN admission_age <= 19 THEN '0-19'
                WHEN admission_age BETWEEN 20 AND 49 THEN '20-49'
                WHEN admission_age BETWEEN 50 AND 79 THEN '50-79'
                ELSE '80+'
            END AS age_group
        FROM patient_ages
    ),
    procedure_counts AS (
        SELECT
            ag.age_group,
            d.long_title AS procedure_name,
            COUNT(*) AS procedure_count,
            ROW_NUMBER() OVER (PARTITION BY ag.age_group ORDER BY COUNT(*) DESC) AS rank
        FROM PROCEDURES_ICD pr
        JOIN age_groups ag ON pr.hadm_id = ag.hadm_id
        JOIN D_ICD_PROCEDURES d ON pr.icd9_code = d.icd9_code
        GROUP BY ag.age_group, d.long_title
    )
    SELECT
        age_group,
        procedure_name,
        procedure_count
    FROM procedure_counts
    WHERE rank <= 3
    ORDER BY age_group, procedure_count DESC;
    """
)

┌───────────┬─────────────────────────────────────────────────────────┬─────────────────┐
│ age_group │                     procedure_name                      │ procedure_count │
│  varchar  │                         varchar                         │      int64      │
├───────────┼─────────────────────────────────────────────────────────┼─────────────────┤
│ 0-19      │ Venous catheterization, not elsewhere classified        │               2 │
│ 0-19      │ Repair of vertebral fracture                            │               1 │
│ 0-19      │ Interruption of the vena cava                           │               1 │
│ 20-49     │ Venous catheterization, not elsewhere classified        │               9 │
│ 20-49     │ Enteral infusion of concentrated nutritional substances │               7 │
│ 20-49     │ Percutaneous abdominal drainage                         │               6 │
│ 50-79     │ Venous catheterization, not elsewhere classified        │              25 │
│ 50-79   

<br> Age is not included in the db so it was calculated by the admit date - date of birth. Age groups were then used to divide the data. The data was aggregated on age groups and procedure name to get the count of each procedure by age group. The top 3 procedures were output.


<br>Venous catherization is common in all age groups. Transfusion of packed cells is also seen in 80+ and 50-79 age groups. Outside of that, there are differences in procedures by age group.

In [42]:
# By Gender
conn.sql(
    """
    WITH icu_ethnicity AS (
        SELECT
            a.ethnicity,
            i.los AS los,
            COUNT(*) OVER (PARTITION BY a.ethnicity) AS ethnic_group_count
        FROM ICUSTAYS i
        JOIN ADMISSIONS a ON i.hadm_id = a.hadm_id
        WHERE i.outtime IS NOT NULL
    ),


    icu_gender AS (
        SELECT
            p.gender,
            i.los AS los,
            COUNT(*) OVER (PARTITION BY p.gender) AS gender_group_count
        FROM ICUSTAYS i
        JOIN PATIENTS p ON i.subject_id = p.subject_id
        WHERE i.outtime IS NOT NULL
    )



    SELECT
        'By Ethnicity' AS analysis_type,
        ethnicity AS category,
        AVG(los) AS avg_los,
        MAX(ethnic_group_count) AS patient_count
    FROM icu_ethnicity
    GROUP BY ethnicity
    HAVING MAX(ethnic_group_count) > 3

    UNION ALL



    SELECT
        'By Gender' AS analysis_type,
        gender AS category,
        AVG(los) AS avg_los,
        MAX(gender_group_count) AS patient_count
    FROM icu_gender
    GROUP BY gender

    ORDER BY analysis_type, avg_los DESC;
    """
)

┌───────────────┬────────────────────────────────┬────────────────────┬───────────────┐
│ analysis_type │            category            │      avg_los       │ patient_count │
│    varchar    │            varchar             │       double       │     int64     │
├───────────────┼────────────────────────────────┼────────────────────┼───────────────┤
│ By Ethnicity  │ BLACK/AFRICAN AMERICAN         │  7.676671428571429 │             7 │
│ By Ethnicity  │ UNKNOWN/NOT SPECIFIED          │  4.925272727272727 │            11 │
│ By Ethnicity  │ WHITE                          │  4.130488043478261 │            92 │
│ By Ethnicity  │ HISPANIC/LATINO - PUERTO RICAN │ 3.2430666666666665 │            15 │
│ By Gender     │ F                              │  5.540071428571428 │            63 │
│ By Gender     │ M                              │  3.513830136986302 │            73 │
└───────────────┴────────────────────────────────┴────────────────────┴───────────────┘

<br>
Due to a small amount of patients of certain ethnicities that had a reported out time, some were excluded. Only ethnicities with more than 3 patients with an out time were included. "los" is the length of ICU stay. This is connected to the gender in PATIENTS or the ethnicity in ADMISSIONS to get the average time in the ICU by gender and race respectively.

<br><br>
This analysis shows that females had a higher average time in ICU than men by around 2 days. Black/African Americans had the most time in ICU, followed by unspecified, White, and Puerto Rican Hispanics.

<br>
<br>

GAI Disclosure: GAI was used to combine two separate queries for ethnicity and gender to have one output